In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://beelike-winkingly-terra.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://beelike-winkingly-terra.ngrok-free.dev


True

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於臺灣新竹縣新豐鄉的私立科技大學。

**歷史沿革**
明新科技大學創立於1966年（民國五十五年），當時以「明新工業專科學校」立案，初期設有機械、土木、工業管理三科。 1993年，更名為「私立明新工商專科學校」。 1997年7月，經教育部核准改制為「明新技術學院」並附設專科部。 最終於2002年8月，獲教育部核准升格為「明新科技大學」。 2018年12月，更名為「明新學校財團法人明新科技大學」。 2022年10月，獲教育部核准通過該校成立56年來第一個博士班——半導體科技博士學位學程。

**地理位置與願景**
學校佔地逾三十公頃，地處新竹縣新豐鄉，交通便利，依傍省道縱貫線並毗鄰中山高速公路。 明新科大的校名取自《大學》「在明明德，在新民，在止於至善」的精義，旨在培養學子具備高尚品德、專業學問與優良技術。 學校的願景是「深耕在地、放眼國際」，並以「培養具實務經驗與人文素養之專業人才」為目標，致力成為「一流產業大學」。

**學術組織與特色**
明新科大現設有半導體學院、工程學院、管理學院、民生學院、人文與設計學院、共同教育學院共六個學院，涵括20個學系、2個學位學程（含1個博士學位學程）及11個碩士班。 學生總數含日間部與進修部約計一萬三千餘人，畢業校友已逾九萬人。

學校積極發展與產業趨勢結合的特色，例如：
*   **半導體學院**：為桃竹苗產業中心及科學園區提供優質的半導體封裝測試人才。
*   **智慧商務與永續發展**：建置「永續智慧商務」教學與實習場域，導入生成式AI與AI專案應用，發展智慧零售、智慧金融、智慧製造與智慧商業，並推動企業溫室氣體管理及產業淨零碳排合作。
*   **跨領域學習與國際視野**：鎖定半導體、AI、元宇宙、風電綠能等前瞻產業，發展MUST四大育才特色，包含多元學習、全球視野、永續經營與技術創新。 此外，學校也積極與國際大學簽訂雙聯學位與合作備忘錄，提升學生國際移動力。 明新科大也是全台率先設立「風力發電學士學位學程」的學校。

明新科技大學以其務實致用的教育理念和與產業緊密結合的特色，致力於培育具備專業技能、團隊精神、人文素養與宏觀服務的科技人才。 在2022年《遠見》雜誌的「企業最愛公私立技職科大

In [ ]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長是呂明峯教授。他於2025年2月1日正式上任，成為明新科技大學的第11任校長。 呂校長期望能以其豐富的半導體教育及產業實務經驗，並透過「四大核心模組」的校務治理策略，帶領明新科大開創技職教育新局。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
